# Demonstrating RCNN With Options In Architecture

- 6/13/18: Coded up the rcnn in keras and with options [SAME, MIX, CNNLSTM, LOAD], we can test out each one and make sure the model compiles.

In [1]:
import sys
import numpy as np 
import os

import keras
import tensorflow as tf

sys.path.append('../../')
import dnn
from dnn.keras_models.nets.rcnn import RCNN
from dnn.keras_models.trainers.rcnn import RCNNTrainer

# import basic plotting
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from dnn.keras_models.nets.cnn import iEEGCNN

# Import magic commands for jupyter notebook 
# - autoreloading a module
# - profiling functions for memory usage and scripts
%load_ext autoreload
%autoreload 2
%load_ext line_profiler
%load_ext memory_profiler

Using TensorFlow backend.
/Users/adam2392/Documents/dnn-unsupervised/.venv/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.23) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
test = np.linspace(1e-5, 1e-3, 10)
print(len(test))

10


In [2]:
# define model
model_params = {
    'name': 'SAME',
    'seqlen': 10,
    'num_classes': 2,
    'imsize': 64,
    'n_colors':4,
}
model = RCNN(**model_params)
print(model)

# define model
model_params = {
    'num_classes': 2,
    'imsize': 64,
    'n_colors':4,
}
convnet = iEEGCNN(**model_params) 
convnet.buildmodel(output=True)

In [3]:
# load in old model
weightsfile = os.path.join('/Users/adam2392/Downloads/exp001/id001_bt/output/loobasecnn_final_weights.h5')
modelfile = os.path.join('/Users/adam2392/Downloads/exp001/id001_bt/output/loobasecnn_model.json')
# model.loadmodel_file(modelfile, weightsfile)

convnet.net.load_weights(weightsfile)
convnet.net.pop()
convnet.net.pop()
convnet.net.pop()
convnet.net.pop()
convnet.net.pop()
convnet.net.pop()
convnet.net.pop()

# print(convnet.net.summary())

In [4]:
# print(convnet.net.summary())
model.loadmodel_net(convnet.net)

# keras.utils.
# test = convnet.net.

In [5]:
model.buildmodel(output=True)
print(model.net)

In [6]:
print(model.net.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 10, 4608)          72064     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               4850688   
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1024)              263168    
_________________________________________________________________
dropout_5 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
__________

In [7]:
print(convnet.net.input.shape)

(?, 64, 64, 4)


# Initialize Trainer and Feed In Data

In [8]:
sys.path.append('../../bin_seq/')
import run_util

# parameters for model
modelname = 'loobasecnn'
num_classes = 2
data_procedure='loo'
# training parameters 
num_epochs = 150
batch_size = 32
seqlen = 20

# Load in the data
traindir = '/Users/adam2392/Downloads/tngpipeline/freq/fft_img/'
testdir = traindir
testpat = 'id001_bt'
procedure = 'loo'

# initialize hpc trainer object
hpcrun = run_util.MarccHPC()

In [9]:
# get the datasets
train_dataset, test_dataset = hpcrun.load_data(traindir, testdir, 
                        seqlen=seqlen,
                        data_procedure=data_procedure, 
                        testpat=testpat)

# get the image size and n_colors from the datasets
imsize = train_dataset.imsize
n_colors = train_dataset.n_colors

2018-06-15 14:05:09,404 - INFO - ReaderSeqDataset - Reading testing data directory /Users/adam2392/Downloads/tngpipeline/freq/fft_img/ 
2018-06-15 14:05:09,405 - INFO - ReaderSeqDataset - Getting test files for id001_bt
2018-06-15 14:05:09,407 - INFO - ReaderSeqDataset - Reading training data directory /Users/adam2392/Downloads/tngpipeline/freq/fft_img/ 
2018-06-15 14:05:09,410 - INFO - ReaderSeqDataset - Found 79 training files and 3 testing files.
2018-06-15 14:05:09,410 - INFO - ReaderSeqDataset - Finished reading in data by directories!
2018-06-15 14:05:09,411 - INFO - ReaderSeqDataset - Loading files from directory!


  0%|          | 0/79 [00:00<?, ?it/s]

(57, 1)
(57, 64, 64, 4)
(539, 64, 64, 4)
0


  1%|▏         | 1/79 [00:00<00:43,  1.79it/s]

1


...

 33%|███▎      | 1/3 [00:00<00:00,  2.61it/s]

1


 67%|██████▋   | 2/3 [00:00<00:00,  3.12it/s]

2


100%|██████████| 3/3 [00:01<00:00,  2.67it/s]


In [10]:
# extract the actual model from the object
model = model.net
print("Image size is {} with {} colors".format(imsize, n_colors))
print("Model is: {}".format(model))
print("Model summary: {}".format(model.summary()))


Image size is 64 with 4 colors
Model is: <keras.engine.sequential.Sequential object at 0x117e55208>
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 10, 4608)          72064     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               4850688   
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1024)              263168    
_________________________________________________________________
dropout_5 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               524800    
__________________________________________

In [ ]:
num_epochs = 1
batch_size = 2

testpatdir = '/Users/adam2392/Downloads/'
expname = 'test'
# train model
trainer = hpcrun.trainmodel(model=model, num_epochs=num_epochs, batch_size=batch_size, 
                    train_dataset=train_dataset, test_dataset=test_dataset,
                    testpatdir=testpatdir,  expname=expname)
# test and save model
trainer = hpcrun.testmodel(trainer, modelname)


2018-06-15 14:06:02,979 - INFO - RCNNTrainer - Logging output data to: /Users/adam2392/Downloads/output
2018-06-15 14:06:02,980 - INFO - RCNNTrainer - Logging experimental data at: /Users/adam2392/Downloads/traininglogs
2018-06-15 14:06:02,981 - INFO - RCNNTrainer - Logging tensorboard data at: /Users/adam2392/Downloads/tensorboard
2018-06-15 14:06:02,982 - INFO - RCNNTrainer - Each training epoch is 22113 steps and each validation is 804 steps.
2018-06-15 14:06:02,982 - INFO - RCNNTrainer - Setting the datasets for training/testing in trainer object!
2018-06-15 14:06:02,983 - INFO - RCNNTrainer - Image size is 64 with 4 colors
Training data:  (20, 64, 64, 4) 20
Testing data:  (20, 64, 64, 4) 20
Class weights are:  [0.55697159 4.88815244]
class imbalance:  11720 22113
Using real-time data augmentation.


In [ ]:
print("DONE")